In [5]:
import os

In [6]:
%pwd

'/Users/liangzhang/Documents/ai/dl-tutorial'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/liangzhang/Documents/ai/dl-tutorial'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
		root_dir: Path
		trained_model_path: Path
		updated_base_model_path: Path
		training_data: Path
		params_epochs: int
		params_batch_size: int
		params_is_augmentation: bool
		params_image_size: list

In [8]:
from template_dl_tutorial.constants import *
from template_dl_tutorial.utils.common import read_yaml, create_directories
import tensorflow as tf

In [9]:
class ConfigurationManager:
	def __init__(
		self,
		config_filePath = CONFIG_FILE_PATH,
		params_filePath = PARAMS_FILE_PATH
	):
		self.config = read_yaml(config_filePath)
		self.params = read_yaml(params_filePath)

		create_directories([self.config.artifacts_root])

	def get_training_config(self):
		training = self.config.training
		prepare_base_model = self.config.prepare_base_model
		params = self.params
		training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'kidney-ct-scan-image')
		create_directories([Path(training.root_dir)])

		training_config = TrainingConfig(
			root_dir=Path(training.root_dir),
			trained_model_path=Path(training.trained_model_path),
			updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
			training_data=Path(training_data),
			params_epochs=params.EPOCHS,
			params_batch_size=params.BATCH_SIZE,
			params_is_augmentation=params.AUGMENTATION,
			params_image_size=params.IMAGE_SIZE
		)

		return training_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [15]:
class Training:
	def __init__(self, config: TrainingConfig):
		self.config = config

	def get_base_model(self):
		self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

	def train_valid_generator(self):
		datagenerator_kwags = dict(
			rescale=1./255,
			validation_split=0.2
		)

		dataflow_kwargs = dict(
			target_size=self.config.params_image_size[:-1],
			batch_size=self.config.params_batch_size,
			interpolation='bilinear'
		)

		valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwags)

		self.valid_generator = valid_datagenerator.flow_from_directory(
			directory=self.config.training_data,
			subset='validation',
			shuffle=False,
			**dataflow_kwargs
		)

		if self.config.params_is_augmentation:
			train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
				rotation_range=40,
				width_shift_range=0.2,
				height_shift_range=0.2,
				shear_range=0.2,
				zoom_range=0.2,
				horizontal_flip=True,
				fill_mode='nearest',
				**datagenerator_kwags
			)
		else:
			train_datagenerator = valid_datagenerator

		self.train_generator = train_datagenerator.flow_from_directory(
			directory=self.config.training_data,
			subset='training',
			shuffle=True,
			**dataflow_kwargs
		)

	@staticmethod
	def save_model(path: Path, model: tf.keras.models.Model):
		model.save(path)

	def train(self):
		self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
		self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

		self.model.fit(
			self.train_generator,
			epochs=self.config.params_epochs,
			steps_per_epoch=self.steps_per_epoch,
			validation_steps=self.validation_steps,
			validation_data=self.valid_generator,
		)
		self.save_model(
			path=self.config.trained_model_path,
			model=self.model
		)


In [16]:
try:
	config = ConfigurationManager()
	training_config = config.get_training_config()
	training = Training(config = training_config)
	training.get_base_model()
	training.train_valid_generator()
	training.train()
except Exception as e:
	raise e

[2024-05-31 18:40:28,172]: INFO: common: yaml file: config/config.yaml loaded successfully
[2024-05-31 18:40:28,175]: INFO: common: yaml file: params.yaml loaded successfully
[2024-05-31 18:40:28,175]: INFO: common: Created directory at artifacts
[2024-05-31 18:40:28,176]: INFO: common: Created directory at artifacts/training
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
23/23 [==============================] - 37s 2s/step - loss: 12.6606 - accuracy: 0.5421 - val_loss: 2.8343 - val_accuracy: 0.4375


/opt/miniconda3/envs/dl/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
